In [1]:
import pandas as pd
import numpy as np
import gc
import re
import MeCab
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim import corpora
model = Word2Vec.load('/tmp/content_based_word2vec_model')

In [2]:
corpus = np.load('corpus_dealonly.npy')
from gensim import corpora
dictionary = corpora.Dictionary(corpus)
dictionary.filter_extremes(no_below=5, no_above=1, keep_n=300000, keep_tokens=None)
corpus_dict = [dictionary.doc2bow(text) for text in corpus]
tfidf = TfidfModel(corpus_dict)

# contents_based itemvec

In [3]:
deal_path = '...'
deal_profile_path = '...'
work_path = '...'
order_path = '...'
df_order = pd.read_csv(work_path + order_path, sep = '\t', compression='gzip')
df_deal = pd.read_csv(work_path + deal_path, sep = '\t', compression='gzip')
df_dealprofile = pd.read_csv(work_path + deal_profile_path, sep = '\t', compression='gzip')

/home/mmde-lab/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9,14,15,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,13,26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_dealprofile_afterprep201712 = pd.read_csv('df_dealprofile_afterprep201712.csv.gz', sep = ',', compression='gzip')
df_dealprofile_afterprep201804 = pd.read_csv('df_dealprofile_afterprep201804.csv.gz', sep = ',', compression='gzip')
df_dealprofile_afterprep201712.deal_description = df_dealprofile_afterprep201712.deal_description.map(lambda x: eval(x))
df_dealprofile_afterprep201804.deal_description = df_dealprofile_afterprep201804.deal_description.map(lambda x: eval(x))

/home/mmde-lab/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,4,10,11,14,18,19,25,26,27,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (11,14,27,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
import datetime
df_order['ins_tm'] = pd.to_datetime(df_order['ins_tm'],format='%Y-%m-%d %H:%M:%S')
df_order['date'] = df_order.ins_tm.dt.date
train_order = df_order.loc[df_order.date <= datetime.date(2017,9,14)]
sell_count = train_order.deal_id.value_counts()
sell = pd.DataFrame(sell_count)
sell = sell.rename(columns= {'deal_id':'sell_count'})
sell = sell.reset_index()
sell.columns = ['deal_id','sell_count']

In [7]:
sell.shape, df_dealprofile_afterprep201712.shape ,df_dealprofile_afterprep201804.shape

((64762, 2), (316024, 36), (73965, 36))

In [9]:
sell_after2 = pd.merge(sell, df_dealprofile_afterprep201712, left_on=['deal_id'], right_on=['deal_id'])

In [18]:
sell_after2 = sell_after2[['deal_id', 'deal_description']]

In [19]:
def calcu_des2vec(raw_des):
    deal_corpus = [dictionary.doc2bow(text) for text in raw_des]
    itemvec = np.zeros([1,200], dtype = float)
    for i in range(len(raw_des)):
        for j in range(len(deal_corpus[i])):
            itemvec += tfidf[deal_corpus][i][j][1]*model[dictionary.get(deal_corpus[i][j][0])]
    return itemvec

In [20]:
sell_after2['dealvec'] = sell_after2.deal_description.map(lambda x: calcu_des2vec(x))

/home/mmde-lab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [21]:
sell_after2.to_csv('df_item_cbwin5_trainset.csv.gz', compression = 'gzip')

In [30]:
sell_corpus = np.load('session_based_corpus_trainset.npy')
deal_dictionary = corpora.Dictionary(sell_corpus)
deal_dictionary.filter_extremes(no_below=5, no_above=1, keep_n=300000, keep_tokens=None)
sell_corpus_dict = [deal_dictionary.doc2bow(text) for text in sell_corpus]
deal_tfidf = TfidfModel(sell_corpus_dict)

In [22]:
train_order.deal_id = train_order.deal_id.map(lambda x: str(x))
grouped_order = train_order.groupby('account_id').apply(lambda x: ' '.join(list(x['deal_id'])))
Input_Data = pd.DataFrame(grouped_order)
Input_Data = Input_Data.reset_index(drop = False)
Input_Data.columns = ['account_id', 'deal_id']
df_user_cbwin5 = Input_Data
df_user_cbwin5.deal_id = df_user_cbwin5.deal_id.map(lambda x: x.split(' '))

/home/mmde-lab/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [49]:
def calcu_user2vec(raw_des):
    user_corpus = deal_dictionary.doc2bow(raw_des)
    uservec = np.zeros([1,200], dtype = float)
    for j in range(len(user_corpus)):
        uservec += tfidf[user_corpus][j][1] * sell_after2.loc[sell_after2.deal_id == int(deal_dictionary.get(user_corpus[0][0]))].dealvec.values[0][0]
    return uservec

In [50]:
df_user_cbwin5['uservec'] = df_user_cbwin5.deal_id.map(lambda x: calcu_user2vec(x))

In [51]:
df_user_cbwin5.to_csv('df_user_cbwin5_trainset.csv.gz', compression = 'gzip')

In [54]:
df_user_cbwin5.uservec[0][0][0]

1.2346416532964213